In [1]:
#https://github.com/amazon-science/efficient-longdoc-classification
from functools import partial
import nltk
import pickle as pk
import torch
from context_enforcement.models.context_enforcer import compute_context_boundary
from context_enforcement.trainers.train_bart3 import model_init
from context_enforcement.data.common import create_text_tokenizer, SmartCollator
from context_enforcement.trainers.common import get_dataset_specified_tasks
from pytorch_lightning import seed_everything

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
seed_everything(1376)

/home/nlplab/anaconda3/envs/development/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 1376
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 1376


1376

In [2]:
z=torch.ones((1,2),dtype=torch.long,device='cuda')
z.device

device(type='cuda', index=0)

In [13]:
configs

Namespace(is_baseline=False, model_base='facebook/bart-base', output_dir='trained_models_sum_boundary/', task_type='xsum', run_id='bart_base_model_context_enforcer_250', eval_steps=1000, learning_rate=5e-05, max_seq_len=800, evaluation_strategy='epoch', save_strategy='epoch', seed=10, lr_scheduler_type='linear', weight_decay=0.0, warmup_ratio=0.25, num_train_epochs=10, save_total_limit=1, per_device_train_batch_size=12, per_device_eval_batch_size=12, gradient_accumulation_steps=4, fp16=True, verbose=False, context_max_len=250)

In [2]:
configs= pk.load(open("trained_models_sum_boundary/bart-base-context-crossed/train_args.ap",'rb'))
context_max_len=configs.context_max_len
context_max_len_list = [context_max_len]#,300,450]
context_sampling_bounds=(0.1, 0.45)

In [3]:
configs.model_base

'facebook/bart-base'

In [3]:
tokenizer = create_text_tokenizer(configs.model_base)

task_dataset_gen = get_dataset_specified_tasks(configs.task_type)

train_dataset = None
eval_dataset = None
test_dataset = None
if task_dataset_gen is not None:
    raw_dataset = task_dataset_gen(tokenizer=tokenizer, )
    train_dataset = raw_dataset['train']
    eval_dataset = raw_dataset['validation']
    test_dataset = raw_dataset['test']

model_builder = model_init(
        vocab_size=len(train_dataset.tokenizer),
        model_base=configs.model_base,
        context_max_len = context_max_len,
        context_sampling_bounds = context_sampling_bounds,
        context_max_len_list= context_max_len_list,#is_baseline=True
    )

Found cached dataset xsum (/home/nlplab/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)
100%|██████████| 3/3 [00:00<00:00, 867.31it/s]


In [4]:
generator = model_builder()
train_model_path = "trained_models_sum_boundary/bart-base-context-crossed/checkpoint-42510/pytorch_model.bin"
state_dict = torch.load(train_model_path)
generator.load_state_dict(state_dict)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Some weights of the model checkpoint at facebook/bart-base were not used when initializing BartForContextualRecovery: ['encoder.layers.1.self_attn.q_proj.bias', 'encoder.layers.1.self_attn_layer_norm.weight', 'encoder.layers.5.self_attn.v_proj.bias', 'encoder.layers.5.self_attn.q_proj.bias', 'encoder.layers.3.self_attn_layer_norm.weight', 'encoder.layers.5.self_attn_layer_norm.bias', 'encoder.layers.1.self_attn.v_proj.weight', 'encoder.layers.3.self_attn.k_proj.weight', 'encoder.layers.1.self_attn.q_proj.weight', 'encoder.layers.3.self_attn.v_proj.bias', 'encoder.layers.3.self_attn.k_proj.bias', 'encoder.layers.3.self_attn.out_proj.weight', 'encoder.layers.1.self_attn.out_proj.bias', 'encoder.layers.5.self_attn.out_proj.weight', 'encoder.layers.3.self_attn_layer_norm.bias', 'encoder.layers.3.self_attn.out_proj.bias', 'encoder.layers.1.self_attn_layer_norm.bias', 'encoder.layers.5.self_attn_layer_norm.weight', 'encoder.layers.5.self_attn.k_proj.weight', 'encoder.layers.5.self_attn.v_pro

In [32]:
te1= test_dataset[692]
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

In [29]:
example= {'document':"""
           Have you been rocking "The Rachel" for the last twenty years? If so, then you're overdue for a trendier haircut that can make the most of your youthful face and features. 
           Check out some stylish magazines or even some celebrity gossip magazines and see what hairstyles are popular these days. 
           You don't have to go for something ultra-trendy if that's not your thing, but getting a haircut that suits you better than your old one can make you look a decade younger.
           """}
te1=test_dataset._process_data(example)
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

In [39]:
#seed_everything(1376)
boundary_sample =  (0.15, 0.65)
seq_len = te1.input_ids[:1024].shape[0]

boundary_width = int(0.21*seq_len) 
context_boundary = compute_context_boundary(seq_len,
                                            context_sampling_bounds=boundary_sample,
                                            context_max_len=boundary_width)
context_boundary,seq_len,context_max_len

((219, 388), 808, 250)

In [40]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=context_boundary,
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=189,
        early_stopping=True,
        use_cache=True,
        num_beams=8,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

['Celtic have appointed Brendan Rodgers as their new manager.']

In [41]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=context_boundary,
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=189,
        early_stopping=True,
        use_cache=True,
        num_beams=8,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

['Celtic have appointed Brendan Rodgers as their new manager.']

In [35]:
test_dataset.tokenizer.decode(te1.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True)

'Celtic have appointed Brendan Rodgers as their new manager.'

In [51]:
te1.input_ids.shape

torch.Size([1187])

In [13]:
print(test_dataset.tokenizer.decode(te1.input_ids,clean_up_tokenization_spaces=True,skip_special_tokens=True))

A judge is considering whether to charge the star over the incident, which was filmed and went viral.
He will spend the night in detention and investigations will continue on Wednesday, a BBC reporter says.
Olomide has denied assault, though he has since apologised for his behaviour.
The Kenyan authorities deported the 60-year-old rumba singer and three of his dancers on Saturday to DR Congo's capital, Kinshasa, following a public outcry over the incident.
The BBC's Poly Muzalia in Kinshasa says police officers arrived at the singer's home early on Tuesday morning and took him into custody.
He was handcuffed, put into a police car and then taken to a court where a judge is considering whether he should be put on trial.
His lawyer, Landry Tanganyi, told the BBC that Olomide, one of Africa's most popular musicians, should not be detained overnight as he was not a flight risk.
However, Olomide left the courthouse under police escort and will spend the night at the police station, our repo

In [22]:
generator.model.context_max_len= 800
generator.model.encoder.context_max_len= 800

In [5]:
from torch.utils.data import DataLoader,SequentialSampler
import tqdm
from context_enforcement.data.common import write_to_file
import evaluate
metrics = evaluate.combine(['bleu','meteor',"rouge"])
def generate():
    test_data_loader = DataLoader(test_dataset,batch_size=12,
                                sampler= SequentialSampler(test_dataset),
                                collate_fn= SmartCollator(
                pad_token_id=train_dataset.tokenizer.pad_token_id,
                max_len=configs.max_seq_len,
                context_max_len=context_max_len,
                context_sampling_bounds=context_sampling_bounds,
            
            ))

    output_summaries =[]
    for batch in tqdm.tqdm(test_data_loader):
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        
        seq_len = b_input_ids.shape[1]

        context_boundary = compute_context_boundary(seq_len,
                                                    context_max_len=110)
        
        boundary_mask =  batch.get("boundary",)
        bb=generator.generate(input_ids=b_input_ids,
                attention_mask=b_input_mask,
                #context_boundary=boundary_mask,
                num_beams=10,
                do_sample=False,
                num_return_sequences=1,
                max_length=140)
        sentences = test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)
        output_summaries+=sentences
    return output_summaries

[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

Token indices sequence length is longer than the specified maximum sequence length for this model (1327 > 1024). Running this sequence through the model will result in indexing errors


In [7]:
os.makedirs("outputs/bart-crossed-context/",exist_ok=True)

In [8]:
context_lens = [100, 200, 350, 720, 650]
outputs = {}
results = {}
for cl in context_lens:
    print(f'Generating for the context length: {cl}')
    generator.model.context_max_len = cl
    generator.model.encoder.context_max_len = cl
    rbase_output = generate()
    outputs[cl] = rbase_output
    
    write_to_file(rbase_output[:len(test_dataset)], 
              f"outputs/bart-crossed-context/best_base_final_{cl}")
    
    scores = metrics.compute(predictions=rbase_output,references=targets)
    print(scores)
    
    results[cl]= scores
    

Generating for the context length: 100


100%|██████████| 945/945 [34:29<00:00,  2.19s/it] 


Done
{'bleu': 0.09728985981551969, 'precisions': [0.40036829449256706, 0.1446563088512241, 0.07350820128713954, 0.040947230830856156], 'brevity_penalty': 0.8466974078222261, 'length_ratio': 0.8573300736494409, 'translation_length': 223734, 'reference_length': 260966, 'meteor': 0.309209022518507, 'rouge1': 0.36110057606137036, 'rouge2': 0.14936732195437305, 'rougeL': 0.2929398773664948, 'rougeLsum': 0.29290562051685765}
Generating for the context length: 200


100%|██████████| 945/945 [40:11<00:00,  2.55s/it]


Done
{'bleu': 0.10443892579045246, 'precisions': [0.41633063147461746, 0.15654621347644604, 0.08147621425077285, 0.046224532913985567], 'brevity_penalty': 0.8343846768188582, 'length_ratio': 0.8466965045254937, 'translation_length': 220959, 'reference_length': 260966, 'meteor': 0.32139797414541343, 'rouge1': 0.374598462597288, 'rouge2': 0.1602632696794728, 'rougeL': 0.3052026232582149, 'rougeLsum': 0.3051469766194579}
Generating for the context length: 350


...

Done
{'bleu': 0.10645165672947572, 'precisions': [0.4220624156914215, 0.16084248767407663, 0.08417190136407067, 0.04791565550342447], 'brevity_penalty': 0.8275550944183794, 'length_ratio': 0.8408451675697217, 'translation_length': 219432, 'reference_length': 260966, 'meteor': 0.32389603939657274, 'rouge1': 0.37812009134128843, 'rouge2': 0.16320197982616513, 'rougeL': 0.3078851645562243, 'rougeLsum': 0.3078703910040433}
Generating for the context length: 720


100%|██████████| 945/945 [1:05:59<00:00,  4.19s/it]


Done
{'bleu': 0.10508792123714251, 'precisions': [0.4205255198314974, 0.15911532994434674, 0.08307806789975586, 0.04735296185906625], 'brevity_penalty': 0.8250273049663323, 'length_ratio': 0.8386877984105209, 'translation_length': 218869, 'reference_length': 260966, 'meteor': 0.32209745515133265, 'rouge1': 0.37621668821754667, 'rouge2': 0.1614728427659432, 'rougeL': 0.3062968848756792, 'rougeLsum': 0.30624375189273667}
Generating for the context length: 650


100%|██████████| 945/945 [1:05:18<00:00,  4.15s/it]


Done
{'bleu': 0.10533236347665792, 'precisions': [0.42049376282745726, 0.15977535131486972, 0.0835742806003029, 0.047421888107724866], 'brevity_penalty': 0.8245777695933528, 'length_ratio': 0.8383046067303787, 'translation_length': 218769, 'reference_length': 260966, 'meteor': 0.3215766959914574, 'rouge1': 0.37601977604613224, 'rouge2': 0.16180846805229285, 'rougeL': 0.30667522805373193, 'rougeLsum': 0.30655150173435614}


Generating for the context length: 200


100%|██████████| 945/945 [08:19<00:00,  1.89it/s]


Done
{'bleu': 0.12346216754890589, 'precisions': [0.4375689199934979, 0.1776036248468456, 0.09612053612814771, 0.05588868746029532], 'brevity_penalty': 0.863722634039824, 'length_ratio': 0.8722170704229669, 'translation_length': 227619, 'reference_length': 260966, 'meteor': 0.35359416748473077, 'rouge1': 0.40248422610308543, 'rouge2': 0.18328221126363148, 'rougeL': 0.33009269996078755, 'rougeLsum': 0.3300614517511684}
Generating for the context length: 720


100%|██████████| 945/945 [10:18<00:00,  1.53it/s]


Done
{'bleu': 0.1257888249451646, 'precisions': [0.4377841740444545, 0.1791020085562382, 0.09760767869694839, 0.05722976542823581], 'brevity_penalty': 0.8695117278704764, 'length_ratio': 0.8773288474360644, 'translation_length': 228953, 'reference_length': 260966, 'meteor': 0.3565514119827842, 'rouge1': 0.4047119203068985, 'rouge2': 0.185856595769887, 'rougeL': 0.332006465804801, 'rougeLsum': 0.33195165056651943}
Generating for the context length: 650


...

Done
{'bleu': 0.1256403068872618, 'precisions': [0.4381767948566864, 0.17878675170700573, 0.09728983506502943, 0.05701369989390655], 'brevity_penalty': 0.8702041264108997, 'length_ratio': 0.8779419541242921, 'translation_length': 229113, 'reference_length': 260966, 'meteor': 0.3563541597980937, 'rouge1': 0.4050806838364527, 'rouge2': 0.18561721029291084, 'rougeL': 0.3319262324656158, 'rougeLsum': 0.33192334002381724}
Generating for the context length: 350


100%|██████████| 945/945 [09:11<00:00,  1.71it/s]


Done
{'bleu': 0.1257626122759007, 'precisions': [0.4382902891885163, 0.17890146449955657, 0.09765710213638545, 0.05715340935321546], 'brevity_penalty': 0.8695030702543955, 'length_ratio': 0.8773211836024616, 'translation_length': 228951, 'reference_length': 260966, 'meteor': 0.3562012415308598, 'rouge1': 0.4047128146167146, 'rouge2': 0.18550529759109602, 'rougeL': 0.33186920040252776, 'rougeLsum': 0.3318646742987025}
Generating for the context length: 100


100%|██████████| 945/945 [07:49<00:00,  2.01it/s]


Done
{'bleu': 0.11964712510107044, 'precisions': [0.4326436377229051, 0.17240038692776577, 0.0924841374122612, 0.05362138236647655], 'brevity_penalty': 0.8627477081454109, 'length_ratio': 0.8713587210594483, 'translation_length': 227395, 'reference_length': 260966, 'meteor': 0.3474841689817138, 'rouge1': 0.39724258276498764, 'rouge2': 0.17816489836175736, 'rougeL': 0.32460150343422817, 'rougeLsum': 0.32467930362884756}


In [11]:
base_output = generate()

100%|██████████| 945/945 [08:28<00:00,  1.86it/s]


In [17]:
generator.model.context_max_len= 800
generator.model.encoder.context_max_len= 800
base_output8 = generate()

100%|██████████| 945/945 [10:04<00:00,  1.56it/s]


In [18]:
generator.model.context_max_len= 600
generator.model.encoder.context_max_len= 600
base_output6 = generate()

100%|██████████| 945/945 [10:02<00:00,  1.57it/s]


In [19]:
generator.model.context_max_len= 450
generator.model.encoder.context_max_len= 450
base_output45 = generate()

100%|██████████| 945/945 [09:30<00:00,  1.66it/s]


In [10]:
#os.makedirs("outputs/multi_context/")

In [25]:
#context_enforcement/data/common.py

write_to_file(base_output8[:len(test_dataset)], 
              "outputs/multi_context/best_base_final_800")

Done


In [14]:
targets[690],base_output[690]#,base_output_8[690],base_output_45[690]#output_summaries2[690],output_summaries[690],output_summaries6[690],output_summaries8[690]

('Musician Koffi Olomide has been taken into custody in the Democratic Republic of Congo, days after he was deported from Kenya for allegedly kicking one of his dancers at an airport in Nairobi.',
 "Uganda's Koffi Olomide has been arrested in the Democratic Republic of Congo over an alleged assault on one of his dancers.")

[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [27]:
scores = metrics.compute(predictions=base_output8,references=targets)
scores

{'bleu': 0.12420563480222273,
 'precisions': [0.4381499894596304,
  0.17908874941070982,
  0.09682579940300837,
  0.05619688787468894],
 'brevity_penalty': 0.864057575751863,
 'length_ratio': 0.8725121280166765,
 'translation_length': 227696,
 'reference_length': 260966,
 'meteor': 0.35420455825818364,
 'rouge1': 0.4026762188767841,
 'rouge2': 0.18481796867562256,
 'rougeL': 0.3305624212280963,
 'rougeLsum': 0.3305498661301927}

In [22]:
scores45 = metrics.compute(predictions=base_output45,
                           references=targets)
scores45

{'bleu': 0.12529167445320155,
 'precisions': [0.43767581217045437,
  0.1782170041154529,
  0.0969340592861464,
  0.056633434208437666],
 'brevity_penalty': 0.8709825773070866,
 'length_ratio': 0.8786316991485481,
 'translation_length': 229293,
 'reference_length': 260966,
 'meteor': 0.35545882674212,
 'rouge1': 0.4042558166707959,
 'rouge2': 0.18471768845866826,
 'rougeL': 0.3318399755379066,
 'rougeLsum': 0.33182453398530987}

In [36]:
{'bleu': 0.13001522827171397,
 'precisions': [0.43264780948765,
  0.17764752271710285,
  0.09726789720912225,
  0.057229256856921935],
 'brevity_penalty': 0.90401138133862,
 'length_ratio': 0.9083367181931746,
 'translation_length': 237045,
 'reference_length': 260966,
 'meteor': 0.3623311725177827,
 'rouge1': 0.40646161409356885,
 'rouge2': 0.18746257090958113,
 'rougeL': 0.3322105156464427,
 'rougeLsum': 20.3334181119671774}

{'bleu': 0.1242693218959997,
 'precisions': [0.4397112710255618,
  0.17984347696769157,
  0.09751670411397062,
  0.05682880711450953],
 'brevity_penalty': 0.8588878671603885,
 'length_ratio': 0.8679674746901895,
 'translation_length': 226510,
 'reference_length': 260966,
 'meteor': 0.3532861459937222,
 'rouge1': 0.4033676736623957,
 'rouge2': 0.18549005001850066,
 'rougeL': 0.3313132654462809,
 'rougeLsum': 0.3313118204210047}

In [17]:
scores

{'bleu': 0.12215108883184916,
 'precisions': [0.42916968255609556,
  0.17216560221044527,
  0.0922525258312802,
  0.0537097051014124],
 'brevity_penalty': 0.8830712157727976,
 'length_ratio': 0.8894032172773465,
 'translation_length': 232104,
 'reference_length': 260966,
 'meteor': 0.35137424074129014,
 'rouge1': 0.398632967632497,
 'rouge2': 0.1799538886903405,
 'rougeL': 0.32609557084990337,
 'rougeLsum': 0.32622241066447516}

In [24]:
scores4

{'bleu': 0.12300035067772054,
 'precisions': [0.4394282548476454,
  0.1790089177800281,
  0.09666579620313663,
  0.056324136455435936],
 'brevity_penalty': 0.8550152182505472,
 'length_ratio': 0.8645762283209307,
 'translation_length': 225625,
 'reference_length': 260966,
 'meteor': 0.35146299353791816,
 'rouge1': 0.4017681486403819,
 'rouge2': 0.18375560820687226,
 'rougeL': 0.32963068346151203,
 'rougeLsum': 0.3296792225053877}

In [18]:
scores5

{'bleu': 0.12202486703008819,
 'precisions': [0.43851006174759005,
  0.17777018935708405,
  0.09596328825046187,
  0.05604684262301944],
 'brevity_penalty': 0.8527557381282802,
 'length_ratio': 0.8626027911681982,
 'translation_length': 225110,
 'reference_length': 260966,
 'meteor': 0.3498951876777049,
 'rouge1': 0.40069971264338244,
 'rouge2': 0.18265325890345102,
 'rougeL': 0.3290151413411355,
 'rougeLsum': 0.3290728012119742}